In [47]:
import os
import pandas as pd
import numpy as np

In [2]:
import pandas as pd

file_path = "export_result_nov18.xlsx"

months = [
    "Dec23","Jan24","Feb24","Mar24","Apr24","May24",
    "Jun24","Jul24","Aug24","Sep24","Oct24","Nov24"
]

xl = pd.ExcelFile(file_path)

for sheet in months:
    df = pd.read_excel(xl, sheet_name=sheet)

    # === TABLE 1 === (A–E → columns 0:5, dynamic length)
    # Detect where table 2 starts ("food_items" in col A)
    food_start = df.index[df.iloc[:, 0].astype(str).str.lower().eq("food_items")]
    if len(food_start) > 0:
        end_t1 = food_start[0]
    else:
        end_t1 = len(df)
    table1 = df.iloc[:end_t1, 0:5].dropna(how="all")

    # === TABLE 2 === (A–D → columns 0:4, starts at "food_items")
    if len(food_start) > 0:
        start_t2 = food_start[0]
        # find next blank row to determine end
        after = df.iloc[start_t2+1:, 0].isna()
        if after.any():
            end_t2 = after.idxmax()
        else:
            end_t2 = len(df)
        table2 = df.iloc[start_t2:end_t2, 0:4].dropna(how="all")
        table2.drop(table2.index[0], inplace=True)
    else:
        table2 = pd.DataFrame()
        table2.drop(table2.index[0], inplace=True)
        
    if table2.shape[1] == 4:
        table2.columns = ['food_items', 'quantity', 'sum_of_portionunitstandard_g', 'sum_of_portionunitstandard_kg']
       
    
    # === ANIMAL & SUMMARY (H–L: cols 7–12) ===
    animal = df.iloc[0:17, 7:12].dropna(how="all")
    animal_name = ['food_items', 'quantity', 'sum_of_portionunitstandard_g',
       'gwp_of_1_kg_food', 'gwp_x_kg']
    animal.columns = animal_name
    
    summary = df.iloc[18:23, 7:12].dropna(how="all")
    summary_name = ['variables', 'mass_kg', 'percentage_kg', 'gwp_total', 'percentage_gwp']
    summary.columns = summary_name

    # === ANIMAL_S & SUMMARY_S (O–S: cols 14–19) ===
    animal_s = df.iloc[0:17, 14:19].dropna(how="all")
    animal_s.columns = animal_name
    summary_s = df.iloc[18:23, 14:19].dropna(how="all")
    summary_s.columns = summary_name

    # === DROP completely empty columns (in case of blanks)
    table1 = table1.dropna(axis=1, how="all")
    table2 = table2.dropna(axis=1, how="all")
    if table2.shape[1] == 3:
        table2.columns = ['food_items', 'sum_of_portionunitstandard_g', 'sum_of_portionunitstandard_kg']
        #print(sheet)

    # === Assign to variables dynamically ===
    suffix = sheet[:3].lower()  # e.g., dec, jan, feb
    globals()[f"{suffix}_table1"]   = table1
    globals()[f"{suffix}_table2"]   = table2
    globals()[f"{suffix}_animal"]   = animal
    globals()[f"{suffix}_summary"]  = summary
    globals()[f"{suffix}_animal_s"] = animal_s
    globals()[f"{suffix}_summary_s"]= summary_s

    #print(f"✅ Extracted all tables for {sheet}")

print("All months processed and loaded into variables.")

All months processed and loaded into variables.


In [53]:
dec_summary

,variables,mass_kg,percentage_kg,gwp_total,percentage_gwp
18,Animal prodcuts on Menu,21572.3804,26.85,155640.955788,55.48
19,Everything else on menu w/ estimates for GWP,49757.09494,61.93,105755.75954,37.7
20,Unestimated Food Mass,9012.43772,11.22,19155.402813,6.83
21,Total,80341.91306,100,280552.118141,100


In [8]:
prefixes = ["dec","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov"]


### 1. The approximate average masses of food ordered by inpatients each month (right now my estimate is 81,000 kg or 81 tonnes)


In [9]:
monthly_masses = []
for p in prefixes:
    summary = globals()[f"{p}_summary"]
    total_mass = summary.loc[summary.variables.str.contains("Total", case=False), "mass_kg"].iloc[0]
    monthly_masses.append(total_mass)

avg_monthly_mass_kg = sum(monthly_masses) / len(monthly_masses)
avg_monthly_mass_tonnes = avg_monthly_mass_kg / 1000

print("1. Average monthly food mass:")
print(f"   {avg_monthly_mass_kg:,.0f} kg   ({avg_monthly_mass_tonnes:.1f} tonnes)")


1. Average monthly food mass:
   81,518 kg   (81.5 tonnes)


In [103]:
avg_monthly_mass_tonnes

81.5181476775

### 2. What the lowest impact food item was on average-- right now I have water at 23kg co2e/month


In [46]:
# 2. LOWEST IMPACT FOOD ITEM (average kg CO2e per month)
# For each month: find item with lowest gwp_x_kg
item_monthly_gwp = {}

for p in prefixes:
    df = globals()[f"{p}_table1"]
    df = df.copy()
    df = df[df['gwp_x_kg'].notna()]  # remove empty rows
    df['food_items'] = df['food_items'].astype(str)

    for _, row in df.iterrows():
        item = row['food_items']
        gwp = row['gwp_x_kg']
        item_monthly_gwp.setdefault(item, []).append(gwp)

# compute average per item
avg_item_gwp = {item: sum(vals)/len(vals) for item, vals in item_monthly_gwp.items() if len(vals) > 2}

lowest_item = min(avg_item_gwp, key=avg_item_gwp.get)
lowest_item_value = avg_item_gwp[lowest_item]

print("2. Lowest impact item across months:")
print(f"   {lowest_item}: {lowest_item_value:.2f} kg CO2e per month")



2. Lowest impact item across months:
   P_tofu_consumed_ON: 16.75 kg CO2e per month


In [31]:
dd = pd.DataFrame(columns=['average', 'len'], index=item_monthly_gwp.keys())
for food in item_monthly_gwp.keys():
    dd.loc[food,'average'] = np.mean(item_monthly_gwp[food])
    dd.loc[food, 'len'] = len(item_monthly_gwp[food])
    #print(food, ":", np.mean(item_monthly_gwp[food]))
    #print()

In [43]:
dd[dd['len']<3]

,average,len
V_cucumber_consumed_ON,5.401667,2
F_orange_consumed_ON,59.434502,2
Supp_Peptamen_TF,2144.2638,1


In [45]:
dd[dd['len']>3].sort_values(by=['average'])#.to_excel('monthly_average.xlsx')

,average,len
P_tofu_consumed_ON,16.752993,12
B_water_at_consumed_ON,23.536734,12
P_chickpea_consumed_ON,28.024541,12
Milk_Soy_Poore&Nemecek,61.225,12
V_greenpea_consumed_ON,74.592405,12
...,...,...
S_icecream_consumed_ON,11051.536341,12
PuddingCup_Proxy_Milk_Eggs_Sugar_Consumed,11364.85879,12
Supp_EnsureChocolate_Water_WheyProtein_Sugar,15363.89776,12
A_milk_consumed_ON,22002.508087,12


In [17]:
np.mean(item_monthly_gwp['B_water_at_consumed_ON'])

np.float64(23.536733520000002)

### 3. What % of the total menu emissions are represented in the top 20 highest emission food items


In [59]:
dd_filtered = dd[dd['len'] > 3]

# Sort by average emissions
dd_sorted = dd_filtered.sort_values(by='average', ascending=False)

# Top 20 food items
top20_items = dd_sorted.head(20).index.tolist()

monthly_shares = []

for p in prefixes:

    # table1 for month
    df = globals()[f"{p}_table1"].copy()
    df['food_items'] = df['food_items'].astype(str)

    # top 20 rows only
    df_top20 = df[df['food_items'].isin(top20_items)]

    # sum emissions for top20
    month_top20_gwp = df_top20['gwp_x_kg'].sum()

    # total GWP from summary table
    summary = globals()[f"{p}_summary"]
    month_total_gwp = summary.loc[summary.variables.str.contains("Total", case=False), 'gwp_total'].iloc[0]

    # monthly share
    share = month_top20_gwp / month_total_gwp
    monthly_shares.append(share)
    
avg_top20_share = np.mean(monthly_shares)
avg_top20_percent = avg_top20_share * 100

print(f"Average percent of emissions from top 20 food items: {avg_top20_percent:.2f}%")


Average percent of emissions from top 20 food items: 79.75%


In [93]:
avg_top20_percent

np.float64(79.7470361487184)

In [92]:
monthly_shares

[np.float64(0.7912559866260526),
 np.float64(0.6892562436972116),
 np.float64(0.8084573846239355),
 np.float64(0.776573066028109),
 np.float64(0.8139902189858688),
 np.float64(0.7904368173618931),
 np.float64(0.7757571967982796),
 np.float64(0.797546917559111),
 np.float64(0.892751832410613),
 np.float64(0.8294316509505254),
 np.float64(0.7867221510082905),
 np.float64(0.8174648717963162)]

In [64]:
avg_top20_percent

np.float64(79.7470361487184)

## 4. What % of total protein sources ordered were plant based (in this included, tofu, veggie burgers, chickpeas, and beans)-- and total quantity of plant-based protein servings versus animal-based protein servings ordered on average in a month.


In [71]:
plant_protein_items = [
    "P_tofu_consumed_ON",
    "VeggieBurg_Soy_Wheat_Fresán",
    "P_chickpea_consumed_ON",
    "P_beans_proxy_chickpea_consumed_ON",
    "Greenbeans_Proxy_chickpeas"
]

animal_protein_items = [
    "A_beef_consumed_ON",
    "A_mixed_meat_consumed_ON",
    "A_pork_consumed_ON",
    "A_salmon_consumed_ON",
    "A_chicken_consumed_ON",
    "Turkey_Proxy_Chicken_Consumed",
    "Tuna_Bianchietal2022",
    "A_egg_consumed_ON",
    "Haddock_Bianchietal2022",
    "Cod_Bianchietal2022"
]


In [73]:
# Prefixes that match your globals
prefixes = ["dec","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov"]
month_labels = ["Dec23","Jan24","Feb24","Mar24","Apr24","May24",
                "Jun24","Jul24","Aug24","Sep24","Oct24","Nov24"]


In [80]:
# -----------------------------
# 1. QUANTITY BASED DATAFRAME
# -----------------------------

qty_results = {
    "plant_qty": {},
    "animal_qty": {},
    "total_plant_animal_qty": {},
    "total_all_foods_qty": {},
    "plant_share_of_protein_pct": {},
    "animal_share_of_protein_pct": {},
    "protein_share_of_all_foods_pct": {}
}

for p, label in zip(prefixes, month_labels):

    # Table 1 always works
    t1 = globals()[f"{p}_table1"].copy()
    t1["food_items"] = t1["food_items"].astype(str)

    # Clean table1 quantity
    if "quantity" in t1.columns:
        t1["quantity"] = pd.to_numeric(t1["quantity"], errors="coerce").fillna(0)
    else:
        t1["quantity"] = 0

    # Table 2 sometimes has no quantity column
    t2 = globals()[f"{p}_table2"].copy() if f"{p}_table2" in globals() else pd.DataFrame()

    if not t2.empty:
        t2["food_items"] = t2["food_items"].astype(str)
        if "quantity" in t2.columns:
            t2["quantity"] = pd.to_numeric(t2["quantity"], errors="coerce").fillna(0)
        else:
            # If no quantity column, set quantity = 0 so we skip table2
            t2["quantity"] = 0
    else:
        # Empty table2
        t2 = pd.DataFrame(columns=["food_items","quantity"])
        t2["quantity"] = 0


    # Plant protein quantities
    plant_qty = (
        t1[t1["food_items"].isin(plant_protein_items)]["quantity"].sum()
        + t2[t2["food_items"].isin(plant_protein_items)]["quantity"].sum()
    )

    # Animal protein quantities
    animal_qty = (
        t1[t1["food_items"].isin(animal_protein_items)]["quantity"].sum()
        + t2[t2["food_items"].isin(animal_protein_items)]["quantity"].sum()
    )

    total_plant_animal_qty = plant_qty + animal_qty

    # Total foods: table1 + table2 (quantity only)
    total_all_foods_qty = t1["quantity"].sum() + t2["quantity"].sum()

    # Store results
    qty_results["plant_qty"][label] = plant_qty
    qty_results["animal_qty"][label] = animal_qty
    qty_results["total_plant_animal_qty"][label] = total_plant_animal_qty
    qty_results["total_all_foods_qty"][label] = total_all_foods_qty

    # Percentages
    if total_plant_animal_qty > 0:
        qty_results["plant_share_of_protein_pct"][label] = 100 * plant_qty / total_plant_animal_qty
        qty_results["animal_share_of_protein_pct"][label] = 100 * animal_qty / total_plant_animal_qty
    else:
        qty_results["plant_share_of_protein_pct"][label] = np.nan
        qty_results["animal_share_of_protein_pct"][label] = np.nan

    if total_all_foods_qty > 0:
        qty_results["protein_share_of_all_foods_pct"][label] = 100 * total_plant_animal_qty / total_all_foods_qty
    else:
        qty_results["protein_share_of_all_foods_pct"][label] = np.nan


# Add yearly averages
for row_name, values in qty_results.items():
    qty_results[row_name]["Average_year"] = pd.Series(values).mean()

# Build quantity dataframe
qty_df = pd.DataFrame(qty_results).T
qty_df = qty_df[month_labels + ["Average_year"]]

print("Quantity based protein summary:")
qty_df


Quantity based protein summary:


,Dec23,Jan24,Feb24,Mar24,Apr24,May24,Jun24,Jul24,Aug24,Sep24,Oct24,Nov24,Average_year
plant_qty,7.372000e+03,7.822000e+03,8914.000000,8666.000000,8.193000e+03,7.882000e+03,7.843000e+03,7861.000000,8.180000e+03,8.385000e+03,8582.000000,8.937000e+03,8.219750e+03
animal_qty,8.620900e+04,8.821700e+04,88275.000000,89349.000000,8.795800e+04,9.024700e+04,9.033200e+04,92998.000000,8.915700e+04,8.808200e+04,91148.000000,8.994100e+04,8.932608e+04
total_plant_animal_qty,9.358100e+04,9.603900e+04,97189.000000,98015.000000,9.615100e+04,9.812900e+04,9.817500e+04,100859.000000,9.733700e+04,9.646700e+04,99730.000000,9.887800e+04,9.754583e+04
total_all_foods_qty,1.014034e+06,1.067207e+06,893408.000000,910265.000000,1.048001e+06,1.054166e+06,1.066668e+06,942374.000000,1.029651e+06,1.028968e+06,925784.000000,1.068089e+06,1.004051e+06
plant_share_of_protein_pct,7.877667e+00,8.144608e+00,9.171820,8.841504,8.520972e+00,8.032284e+00,7.988796e+00,7.794049,8.403793e+00,8.692092e+00,8.605234,9.038411e+00,8.425936e+00
animal_share_of_protein_pct,9.212233e+01,9.185539e+01,90.828180,91.158496,9.147903e+01,9.196772e+01,9.201120e+01,92.205951,9.159621e+01,9.130791e+01,91.394766,9.096159e+01,9.157406e+01
protein_share_of_all_foods_pct,9.228586e+00,8.999098e+00,10.878456,10.767743,9.174705e+00,9.308686e+00,9.203895e+00,10.702651,9.453397e+00,9.375121e+00,10.772491,9.257468e+00,9.760192e+00


In [89]:
qty_df.to_clipboard()

In [87]:
# -----------------------------
# 2. GWP BASED DATAFRAME
# (unchanged from before)
# -----------------------------

gwp_results = {
    "plant_gwp": {},
    "animal_gwp": {},
    "total_plant_animal_gwp": {},
    "total_all_foods_gwp": {},
    "plant_share_of_protein_gwp_pct": {},
    "animal_share_of_protein_gwp_pct": {},
    "protein_share_of_all_foods_gwp_pct": {}
}

for p, label in zip(prefixes, month_labels):

    t1 = globals()[f"{p}_table1"].copy()
    t1["food_items"] = t1["food_items"].astype(str)
    t1["gwp_x_kg"] = pd.to_numeric(t1["gwp_x_kg"], errors="coerce").fillna(0)

    plant_gwp = t1[t1["food_items"].isin(plant_protein_items)]["gwp_x_kg"].sum()
    animal_gwp = t1[t1["food_items"].isin(animal_protein_items)]["gwp_x_kg"].sum()
    total_plant_animal_gwp = plant_gwp + animal_gwp

    summary = globals()[f"{p}_summary"]
    total_all_foods_gwp = pd.to_numeric(
        summary.loc[summary["variables"].astype(str).str.contains("Total", case=False), "gwp_total"],
        errors="coerce"
    ).iloc[0]

    gwp_results["plant_gwp"][label] = plant_gwp
    gwp_results["animal_gwp"][label] = animal_gwp
    gwp_results["total_plant_animal_gwp"][label] = total_plant_animal_gwp
    gwp_results["total_all_foods_gwp"][label] = total_all_foods_gwp

    if total_plant_animal_gwp > 0:
        gwp_results["plant_share_of_protein_gwp_pct"][label] = 100 * plant_gwp / total_plant_animal_gwp
        gwp_results["animal_share_of_protein_gwp_pct"][label] = 100 * animal_gwp / total_plant_animal_gwp
    else:
        gwp_results["plant_share_of_protein_gwp_pct"][label] = np.nan
        gwp_results["animal_share_of_protein_gwp_pct"][label] = np.nan

    gwp_results["protein_share_of_all_foods_gwp_pct"][label] = (
        100 * total_plant_animal_gwp / total_all_foods_gwp
        if total_all_foods_gwp > 0 else np.nan
    )

# Add averages
for row_name, values in gwp_results.items():
    gwp_results[row_name]["Average_year"] = pd.Series(values).mean()

# GWP dataframe
gwp_df = pd.DataFrame(gwp_results).T
gwp_df = gwp_df[month_labels + ["Average_year"]]

print("GWP based protein summary (kg CO2e):")
gwp_df

GWP based protein summary (kg CO2e):


,Dec23,Jan24,Feb24,Mar24,Apr24,May24,Jun24,Jul24,Aug24,Sep24,Oct24,Nov24,Average_year
plant_gwp,605.867105,643.123450,708.636818,679.309589,688.619901,685.135502,655.012478,673.352212,674.678383,700.674622,751.074201,750.670231,684.679541
animal_gwp,104946.490807,110656.915452,107097.480092,102039.727248,117664.455367,104527.931176,100671.595952,106684.942037,132223.424603,125112.149860,106057.558018,127237.122489,112076.649425
total_plant_animal_gwp,105552.357911,111300.038902,107806.116910,102719.036837,118353.075268,105213.066678,101326.608430,107358.294249,132898.102986,125812.824482,106808.632219,127987.792720,112761.328966
total_all_foods_gwp,280552.118141,348712.848043,284173.642037,301420.549161,306021.572844,290691.226026,289773.896162,291497.934422,294511.518677,305135.481654,291770.745043,316982.158445,300103.640888
plant_share_of_protein_gwp_pct,0.573997,0.577829,0.657325,0.661328,0.581835,0.651189,0.646437,0.627201,0.507666,0.556918,0.703196,0.586517,0.610953
animal_share_of_protein_gwp_pct,99.426003,99.422171,99.342675,99.338672,99.418165,99.348811,99.353563,99.372799,99.492334,99.443082,99.296804,99.413483,99.389047
protein_share_of_all_foods_gwp_pct,37.623084,31.917390,37.936705,34.078313,38.674749,36.194098,34.967473,36.829864,45.124925,41.231791,36.607040,40.376971,37.630200


In [95]:
gwp_df.to_clipboard()